<b>4.1. Explain del Pipeline</b>
<br>A continuaciòn se muestra el diagrama DAG del flujo de transformaciones del conjunto de datos aplicado en el consumer de Kafka:<br>
- Scan ExistingRDD: Selecciona el conjunto de datos transaccional principal del proyecto proveniente del KafkaConsumer.
- FileScan CSV taxi_zone_lookup: Lectura de los conjuntos de datos que contienen las ubicaciones que alimentan el dataset con el LocationID.
- Project LocationID, Zone (pickup_zone)/(delivery_zone): Se realiza el cambio de nombre de la columna zone para ambos dataframe especìficamente.
- Filter (atleastnnonnulls): Limpieza de nulos en el conjunto de datos principal.
- Project + cálculo uber_sales (fare + tips - driver_pay): Se realiza el càlculo de columnas para definir la comisiòn de la plataforma.
- BroadcastHashJoin (LeftOuter) PULocationID = LocationID BuildRight (pickup_zone): Cruce de dataframes durante la transmisiòn para obtener el lugar de inicio del viaje.
- BroadcastHashJoin (LeftOuter) DOLocationID = LocationID BuildRight (delivery_zone): Cruce de tablas durante la transmisiòn para obtener el lugar de finalizaciòn del viaje.

                                      ┌──────────────────────────────┐
                                      │        Project (final)       │
                                      │ index_trip, fechas, zonas,   │
                                      │ uber_sales, pickup_zone,     │
                                      │ delivery_zone                │
                                      └───────────────┬──────────────┘
                                                      │
                           ┌──────────────────────────▼──────────────────────────┐
                           │        BroadcastHashJoin (LeftOuter)                │
                           │        DOLocationID = LocationID                    │
                           │        BuildRight (delivery_zone)                   │
                           └───────────────┬───────────────────────┬─────────────┘
                                           │                       │
                   ┌───────────────────────▼──────────────────┐    │
                   │        Project (con pickup_zone)         │    │
                   └───────────────┬──────────────────────────┘    │
                                   │                               │
        ┌──────────────────────────▼──────────────────────────┐    │
        │        BroadcastHashJoin (LeftOuter)                │    │
        │        PULocationID = LocationID                    │    │
        │        BuildRight (pickup_zone)                     │    │
        └───────────────┬───────────────────────┬─────────────┘    │
                        │                       │                  │
      ┌─────────────────▼─────────────────┐     │                  │
      │   Project + cálculo uber_sales    │     │                  │
      │   (fare + tips - driver_pay)      │     │                  │
      └───────────────┬───────────────────┘     │                  │
                      │                         │                  │
      ┌───────────────▼───────────────────┐     │                  │
      │   Filter (atleastnnonnulls)       │     │                  │
      └───────────────┬───────────────────┘     │                  │
                      │                         │                  │
      ┌───────────────▼───────────────────┐     │                  │
      │        Scan ExistingRDD           │     │                  │
      │        (trips dataset)            │     │                  │
      └───────────────────────────────────┘     │                  │
                                                │                  │
                        ┌───────────────────────▼─────────────┐    │
                        │   BroadcastExchange                 │    │
                        └───────────────┬─────────────────────┘    │
                                        │                          │
                        ┌───────────────▼─────────────────────┐    │
                        │ Project LocationID, Zone            │    │
                        │ (pickup_zone)                       │    │
                        └───────────────┬─────────────────────┘    │
                                        │                          │
                        ┌───────────────▼─────────────────────┐    │
                        │ FileScan CSV taxi_zone_lookup       │    │
                        │ (pickup)                            │    │
                        └─────────────────────────────────────┘    │
                                                                   │
                                       ┌───────────────────────────▼─────────────┐
                                       │   BroadcastExchange                     │
                                       └───────────────┬─────────────────────────┘
                                                       │
                                       ┌───────────────▼─────────────────────────┐
                                       │ Project LocationID, Zone                │
                                       │ (delivery_zone)                         │
                                       └───────────────┬─────────────────────────┘
                                                       │
                                       ┌───────────────▼─────────────────────────┐
                                       │ FileScan CSV taxi_zone_lookup           │
                                       │ (delivery)                              │
                                       └─────────────────────────────────────────┘
